<a href="https://colab.research.google.com/github/Aatti13/agrihack/blob/main/agrihack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agro Hack

The following notebook is the basic pipeline for Disease Prediction in Crops using the YOLO object-detection algorithm.

The following pipeline consists of 5 stages (but 4 key stages that shall be highlighted in **BOLD**):
  1. Obtaining the original Dataset
  2. **OpenCV CVAT offline setup**
  3. **Performing Annotation on both datasets**
  4. **Image Augmentation techniques to ensure all bases are covered**
  5. **Multimodal setup invoving YOLOv8, YOLOv8-seg and TabNet**
  6. Results and Conclusion

This document contains a comprehensive explanation of each step.

---
## Stage-1: Obtaining the Datasets

The first step is to download the initial datasets:
- Crop Disease Dataset: [Crop-dataset](https://drive.google.com/drive/folders/1-mgBbJnEOSewrKsgmn4aDWbHKaUAeGrD?usp=sharing)
- Insect Dataset: [Insect Dataset](https://drive.google.com/drive/folders/1szQoqBQdBH4xm8Yo_FxQ34xVGpJOnTgl?usp=sharing)

Both datasets contain 50 images each

---

## Stage-2: Running CVAT offline

CVAT or *Computer Vision Annotation Tool* is a tool developed by OpenCV that helps us annotate image data using mutiple schemes, be it bounding box, segmentation or any other type.

There are a few pre-requisites before we get to the installation, they are:
<br>

  ### **Install Git on your Device**:
  Go to the official website: [Install Git](https://git-scm.com/downloads)

  ### **Install Docker**:
  Visit the Docker Website: [Install Docker](https://www.docker.com/)

After these have been installed we can now move on to the next bit

  ### **Clone the CVAT GitHub Repo**
  Visit the cvat github page: [CVAT GitHub](https://github.com/cvat-ai/cvat)
  <br><br>
  Copy the git link and in you powershell/Commpand prompt and type the following command:<br>
  `git clone https://github.com/cvat-ai/cvat.git`

  ### **Run a Docker container**
  

---

## Stage-3: Perform Annotation on Both Datasets

We are supposed to perform 2 different annotation schemes on both the datasets:

<table>
  <tr>
    <th>Dataset</th>
    <th>Annotation Type</th>
  </tr>
  <tr>
    <td>Crop Disease Dataset</td>
    <td>Segmentation Annotation</td>
  </tr>
  <tr>
    <td>Crop Insect Dataset</td>
    <td>Bounding-Box Annotation</td>
  </tr>
</table>

---

## Stage-4: Image Augmentation

We are supposed to perform the following Augmentations:

- Normalization
- Rotation
- Flipping
- Colour Jittering
- Contrast Enhancement

In [ ]:
!pip install opencv-python pandas pillow albumentations ultralytics pytorch-tabnet scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [ ]:

import os
import sys
import json
import logging
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Any, Optional

warnings.filterwarnings('ignore')

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

from ultralytics import YOLO
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('multimodal_pipeline.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

In [39]:
class DataAugmentation:
    def __init__(self, no_of_augmented_images_per_original=5):
        self.num_augmented_images = no_of_augmented_images_per_original
        print(f"DataAugmenter initialized. Will create {self.num_augmented_images} versions per original image.")

        # Updated input directories based on user's request
        self.crop_inp_dir = '/content/drive/MyDrive/Colab Datasets/disease_train'
        self.insect_inp_dir = '/content/drive/MyDrive/Colab Datasets/Insects_train'

        # Output directories
        self.crop_out_dir = 'crop_aug'
        self.insect_out_dir = 'insect_aug'

        self.segment_transform = self._get_segment_pipeline()
        self.bbox_transform = self._get_bbox_pipeline()


    def _get_segment_pipeline(self):
        """Define augmentation pipeline for segmented images (masks)"""
        return A.Compose([
            A.Rotate(limit=45, p=0.8),
            A.HorizontalFlip(p=1.0),
            A.VerticalFlip(p=1.0),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=1.0),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),
            A.CLAHE(clip_limit=4.0, p=1),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ], p=1.0, is_check_shapes=False) # Apply all transformations with probability 1.0 and disable shape check


    def _get_bbox_pipeline(self):
        """Define augmentation pipeline for bounding box annotations"""
        return A.Compose([
            A.Rotate(limit=45, p=1.0),
            A.HorizontalFlip(p=1.0),
            A.VerticalFlip(p=1.0),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=1.0),
            A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=1.0),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']), p=1.0) # Removed check_validity


    def augment_segmented_dataset(self):
        """Augment the segmented (disease) dataset"""
        print(f"\n--- Starting Segmentation Augmentation for {self.crop_inp_dir} ---")
        image_dir = os.path.join(self.crop_inp_dir, 'images', 'Train')
        label_dir = os.path.join(self.crop_inp_dir, 'labels', 'Train')

        output_image_dir = os.path.join(self.crop_out_dir, 'images')
        output_label_dir = os.path.join(self.crop_out_dir, 'labels')

        os.makedirs(output_image_dir, exist_ok=True)
        os.makedirs(output_label_dir, exist_ok=True)

        image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

        for filename in image_files:
            image_path = os.path.join(image_dir, filename)
            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_path = os.path.join(label_dir, label_filename)

            if not os.path.exists(label_path):
                print(f"Warning: Label file not found for {filename}. Skipping.")
                continue

            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Read segmentation masks (assuming YOLO segmentation format)
            masks = []
            class_labels = []
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    # The rest of the parts are the polygon points for segmentation
                    points = [float(x) for x in parts[1:]]
                    # Reshape points to (num_points, 2)
                    polygon = np.array(points).reshape(-1, 2)

                    # Create a blank mask for the current image size
                    mask = np.zeros(image.shape[:2], dtype=np.uint8)
                    # Convert normalized polygon points to absolute pixel coordinates
                    h, w = image.shape[:2]
                    abs_polygon = (polygon * np.array([w, h])).astype(np.int32)
                    # Draw the polygon on the mask
                    cv2.fillPoly(mask, [abs_polygon], 1) # Fill with 1 for the object

                    masks.append(mask)
                    class_labels.append(class_id)

            masks = np.array(masks).transpose(1, 2, 0) # Convert to (H, W, N) format


            for i in range(self.num_augmented_images):
                try:
                    augmented = self.segment_transform(image=image, masks=masks)
                    aug_image = augmented['image']
                    aug_masks = augmented['masks']

                    # Convert normalized image back to uint8 for saving
                    aug_image = (aug_image * 255).astype(np.uint8)
                    aug_image = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)

                    new_filename_base = f"{os.path.splitext(filename)[0]}_aug_{i+1}"
                    new_image_path = os.path.join(output_image_dir, f"{new_filename_base}.jpg")
                    new_label_path = os.path.join(output_label_dir, f"{new_filename_base}.txt")

                    cv2.imwrite(new_image_path, aug_image)

                    # Convert augmented masks back to YOLO segmentation format
                    with open(new_label_path, 'w') as f:
                        processed_masks_count = 0
                        for mask_idx in range(aug_masks.shape[-1]):
                            if processed_masks_count >= len(class_labels):
                                # Stop processing if we have already written labels for the original number of objects
                                break

                            binary_mask = aug_masks[:, :, mask_idx]
                            # Find contours from the binary mask
                            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                            # Filter and sort contours by area (descending)
                            valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 10] # Filter small contours
                            valid_contours = sorted(valid_contours, key=cv2.contourArea, reverse=True)

                            if valid_contours:
                                # Take the largest contour (assuming one main object per mask instance after filtering)
                                largest_contour = valid_contours[0]

                                # Convert contour points to normalized YOLO format
                                h, w = aug_image.shape[:2]
                                normalized_points = largest_contour.squeeze().astype(np.float32)

                                # Ensure normalized_points is not empty before dividing
                                if normalized_points.size > 0:
                                    # Ensure normalized_points has shape (N, 2)
                                    if normalized_points.ndim == 1:
                                        normalized_points = normalized_points.reshape(-1, 2)

                                    normalized_points[:, 0] /= w
                                    normalized_points[:, 1] /= h
                                    # Flatten the points
                                    flat_points = normalized_points.flatten().tolist()

                                    # Get the corresponding class label based on the original index
                                    # This assumes order is mostly preserved for largest contours
                                    class_id = class_labels[processed_masks_count]

                                    # Write to file
                                    line = f"{class_id} {' '.join(map(str, flat_points))}\n"
                                    f.write(line)
                                    processed_masks_count += 1
                                else:
                                     print(f"Warning: No valid contour points found for mask {mask_idx} in {new_filename_base}. Skipping.")


                except Exception as e:
                    print(f"Error augmenting {filename} (segmentation), augmentation {i+1}: {e}")
                    continue


        print(f"Segmentation augmentation complete. Augmented data saved to {self.crop_out_dir}")


    def augment_bbox_dataset(self):
        """Augment the bounding box (insect) dataset"""
        print(f"\n--- Starting Bounding Box Augmentation for {self.insect_inp_dir} ---")
        image_dir = os.path.join(self.insect_inp_dir, 'images', 'Train') # Assuming similar structure
        label_dir = os.path.join(self.insect_inp_dir, 'labels', 'Train') # Assuming similar structure

        output_image_dir = os.path.join(self.insect_out_dir, 'images')
        output_label_dir = os.path.join(self.insect_out_dir, 'labels')

        os.makedirs(output_image_dir, exist_ok=True)
        os.makedirs(output_label_dir, exist_ok=True)

        image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

        for filename in image_files:
            image_path = os.path.join(image_dir, filename)
            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_path = os.path.join(label_dir, label_filename)

            if not os.path.exists(label_path):
                print(f"Warning: Label file not found for {filename}. Skipping.")
                continue

            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            bboxes = []
            class_labels = []
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    # YOLO bbox format: class_id x_center y_center width height (normalized)
                    coords = [float(x) for x in parts[1:]]
                    bboxes.append(coords)
                    class_labels.append(class_id)

            for i in range(self.num_augmented_images):
                # Initialize variables before the try block where they might be assigned
                aug_bboxes = None
                aug_class_labels = None
                try:
                    augmented = self.bbox_transform(image=image, bboxes=bboxes, class_labels=class_labels)
                    aug_image = augmented['image']
                    aug_bboxes = augmented['bboxes']
                    aug_class_labels = augmented['class_labels']

                    # Check if augmentation was successful and returned bounding boxes
                    if aug_bboxes is not None and aug_class_labels is not None:
                        # Manually clip bounding box coordinates to the [-1.0, 1.0] range
                        clipped_aug_bboxes = []
                        for bbox in aug_bboxes:
                            clipped_bbox = [
                                max(-1.0, min(1.0, bbox[0])),
                                max(-1.0, min(1.0, bbox[1])),
                                max(-1.0, min(1.0, bbox[2])),
                                max(-1.0, min(1.0, bbox[3]))
                            ]
                            clipped_aug_bboxes.append(clipped_bbox)


                        # Convert normalized image back to uint8 for saving
                        aug_image = (aug_image * 255).astype(np.uint8)
                        aug_image = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)

                        new_filename_base = f"{os.path.splitext(filename)[0]}_aug_{i+1}"
                        new_image_path = os.path.join(output_image_dir, f"{new_filename_base}.jpg")
                        new_label_path = os.path.join(output_label_dir, f"{new_filename_base}.txt")

                        cv2.imwrite(new_image_path, aug_image)

                        with open(new_label_path, 'w') as f:
                            for bbox, label in zip(clipped_aug_bboxes, aug_class_labels):
                                # Format back to YOLO string: class_id x_center y_center width height
                                yolo_line = f"{label} {' '.join(map(str, bbox))}\n"
                                f.write(yolo_line)
                    else:
                        print(f"Warning: Bounding box augmentation failed for {filename}, augmentation {i+1}. Skipping saving results.")


                except Exception as e:
                    print(f"Error augmenting {filename} (bbox), augmentation {i+1}: {e}")
                    continue


        print(f"Bounding box augmentation complete. Augmented data saved to {self.insect_out_dir}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class YOLODiseaseDetector:
    """YOLOv8s-seg model for crop disease detection"""

    def __init__(self, model_path: Optional[str] = None, confidence: float = 0.25):
        self.confidence = confidence
        self.model_path = model_path or 'yolov8s-seg.pt'
        self.model = None
        self.class_names = []
        self.load_model()

    def load_model(self):
        """Load YOLOv8 segmentation model"""
        try:
            if os.path.exists(self.model_path):
                self.model = YOLO(self.model_path)
                logger.info(f"Loaded custom model from {self.model_path}")
            else:
                self.model = YOLO('yolov8s-seg.pt')
                logger.info("Loaded pretrained YOLOv8s-seg model")

            if hasattr(self.model, 'names'):
                self.class_names = list(self.model.names.values())

        except Exception as e:
            logger.error(f"Error loading YOLOv8s-seg model: {e}")
            raise

    def train(self, data_yaml_path: str, epochs: int = 100, imgsz: int = 640):
        """Train YOLOv8 model on disease dataset"""
        try:
            results = self.model.train(
                data=data_yaml_path,
                epochs=epochs,
                imgsz=imgsz,
                batch=16,
                device='auto',
                patience=20,
                save=True,
                cache=True,
                workers=4,
                optimizer='AdamW',
                lr0=0.001,
                weight_decay=0.0005,
                warmup_epochs=3,
                mosaic=1.0,
                mixup=0.1,
                copy_paste=0.3
            )
            logger.info("Disease detection model training completed")
            return results
        except Exception as e:
            logger.error(f"Error training disease model: {e}")
            raise

    def detect(self, image_path: str) -> Dict[str, Any]:
        """Detect diseases in crop image"""
        try:
            results = self.model(image_path, conf=self.confidence, verbose=False)

            detection_results = {
                'diseases_detected': [],
                'confidence_scores': [],
                'bounding_boxes': [],
                'segmentation_masks': [],
                'disease_present': False
            }

            for result in results:
                if result.boxes is not None and len(result.boxes) > 0:
                    detection_results['disease_present'] = True

                    for box, conf, cls in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
                        class_name = self.class_names[int(cls)] if self.class_names else f"disease_{int(cls)}"
                        detection_results['diseases_detected'].append(class_name)
                        detection_results['confidence_scores'].append(float(conf))
                        detection_results['bounding_boxes'].append(box.cpu().numpy().tolist())

                if hasattr(result, 'masks') and result.masks is not None:
                    masks = result.masks.data.cpu().numpy()
                    detection_results['segmentation_masks'] = masks.tolist()

            logger.info(f"Disease detection completed. Found {len(detection_results['diseases_detected'])} diseases")
            return detection_results

        except Exception as e:
            logger.error(f"Error in disease detection: {e}")
            return {'diseases_detected': [], 'confidence_scores': [], 'disease_present': False}

In [ ]:
class YOLOInsectDetector:
    """YOLOv8s model for crop insect detection"""

    def __init__(self, model_path: Optional[str] = None, confidence: float = 0.25):
        self.confidence = confidence
        self.model_path = model_path or 'yolov8s.pt'
        self.model = None
        self.class_names = []
        self.load_model()

    def load_model(self):
        """Load YOLOv8 detection model"""
        try:
            if os.path.exists(self.model_path):
                self.model = YOLO(self.model_path)
                logger.info(f"Loaded custom insect model from {self.model_path}")
            else:
                self.model = YOLO('yolov8s.pt')
                logger.info("Loaded pretrained YOLOv8s model")

            if hasattr(self.model, 'names'):
                self.class_names = list(self.model.names.values())

        except Exception as e:
            logger.error(f"Error loading YOLOv8s insect model: {e}")
            raise

    def train(self, data_yaml_path: str, epochs: int = 100, imgsz: int = 640):
        """Train YOLOv8 model on insect dataset"""
        try:
            results = self.model.train(
                data=data_yaml_path,
                epochs=epochs,
                imgsz=imgsz,
                batch=16,
                device='auto',
                patience=20,
                save=True,
                cache=True,
                workers=4,
                optimizer='AdamW',
                lr0=0.001,
                weight_decay=0.0005,
                warmup_epochs=3,
                mosaic=1.0,
                mixup=0.1
            )
            logger.info("Insect detection model training completed")
            return results
        except Exception as e:
            logger.error(f"Error training insect model: {e}")
            raise

    def detect(self, image_path: str) -> Dict[str, Any]:
        """Detect insects in crop image"""
        try:
            results = self.model(image_path, conf=self.confidence, verbose=False)

            detection_results = {
                'insects_detected': [],
                'confidence_scores': [],
                'bounding_boxes': [],
                'insect_present': False
            }

            for result in results:
                if result.boxes is not None and len(result.boxes) > 0:
                    detection_results['insect_present'] = True

                    for box, conf, cls in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
                        class_name = self.class_names[int(cls)] if self.class_names else f"insect_{int(cls)}"
                        detection_results['insects_detected'].append(class_name)
                        detection_results['confidence_scores'].append(float(conf))
                        detection_results['bounding_boxes'].append(box.cpu().numpy().tolist())

            logger.info(f"Insect detection completed. Found {len(detection_results['insects_detected'])} insects")
            return detection_results

        except Exception as e:
            logger.error(f"Error in insect detection: {e}")
            return {'insects_detected': [], 'confidence_scores': [], 'insect_present': False}


In [14]:
class TabNetPredictor:
    """TabNet model for symptom-based classification"""

    def __init__(self, model_type: str = 'disease'):
        self.model_type = model_type
        self.model = None
        self.feature_columns = []
        self.is_trained = False
        self.scaler = None

    def prepare_data(self, csv_path: str) -> Tuple[np.ndarray, np.ndarray]:
        """Prepare data from CSV file"""
        try:
            df = pd.read_csv(csv_path)
            logger.info(f"Loaded {len(df)} samples from {csv_path}")

            if 'target' in df.columns:
                X = df.drop('target', axis=1)
                y = df['target']
            else:
                # Assuming the last column is the target if not explicitly named
                X = df.iloc[:, :-1]
                y = df.iloc[:, -1]

            self.feature_columns = X.columns.tolist()

            for col in X.columns:
                if X[col].dtype == 'object':
                    X[col] = pd.Categorical(X[col]).codes

            if y.dtype == 'object':
                y = pd.Categorical(y).codes

            return X.values.astype(np.float32), y.values.astype(np.int64)

        except Exception as e:
            logger.error(f"Error preparing data: {e}")
            raise

    def train(self, csv_path: str, test_size: float = 0.2, max_epochs: int = 10):
        """Train TabNet model"""
        try:
            X, y = self.prepare_data(csv_path)

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42, stratify=y
            )

            self.scaler = StandardScaler()
            X_train = self.scaler.fit_transform(X_train)
            X_test = self.scaler.transform(X_test)

            self.model = TabNetClassifier(
                n_d=32, n_a=32, n_steps=5, gamma=1.5, lambda_sparse=1e-3,
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                mask_type='entmax',
                scheduler_params=dict(step_size=50, gamma=0.9),
                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                verbose=1, device_name='auto'
            )

            self.model.fit(
                X_train, y_train,
                eval_set=[(X_test, y_test)],
                max_epochs=max_epochs, patience=1000, # Increased patience to effectively disable early stopping
                batch_size=256, virtual_batch_size=128,
                num_workers=0, drop_last=False,
                eval_metric=['accuracy']
            )

            self.is_trained = True

            train_preds = self.model.predict(X_train)
            test_preds = self.model.predict(X_test)

            train_accuracy = accuracy_score(y_train, train_preds)
            test_accuracy = accuracy_score(y_test, test_preds)

            logger.info(f"TabNet {self.model_type} model training completed")
            logger.info(f"Train Accuracy: {train_accuracy:.4f}")
            logger.info(f"Test Accuracy: {test_accuracy:.4f}")

            return {
                'train_accuracy': train_accuracy,
                'test_accuracy': test_accuracy,
                'feature_importance': self.model.feature_importances_.tolist()
            }

        except Exception as e:
            logger.error(f"Error training TabNet model: {e}")
            raise

    def predict(self, symptoms: Dict[str, Any]) -> Dict[str, Any]:
        """Predict based on farmer's symptom input"""
        try:
            if not self.is_trained:
                logger.warning("Model not trained yet")
                # Return a default prediction
                return {'prediction': 0, 'confidence': 0.0, 'present': False, 'probabilities': [1.0, 0.0]}

            feature_vector = []
            for col in self.feature_columns:
                if col in symptoms:
                    value = symptoms[col]
                    if isinstance(value, str):
                        value = 1 if value.lower() in ['yes', 'y', '1', 'true'] else 0
                    else:
                        value = float(value)
                    feature_vector.append(value)
                else:
                    feature_vector.append(0)

            feature_vector = np.array(feature_vector).reshape(1, -1).astype(np.float32)
            if self.scaler:
                feature_vector = self.scaler.transform(feature_vector)

            prediction = self.model.predict(feature_vector)[0]
            probabilities = self.model.predict_proba(feature_vector)[0]
            confidence = float(np.max(probabilities))

            result = {
                'prediction': int(prediction),
                'confidence': confidence,
                'present': bool(prediction > 0),
                'probabilities': probabilities.tolist()
            }

            logger.info(f"TabNet {self.model_type} prediction: {result}")
            return result

        except Exception as e:
            logger.error(f"Error in TabNet prediction: {e}")
            return {'prediction': 0, 'confidence': 0.0, 'present': False, 'probabilities': [1.0, 0.0]}

In [ ]:
class MultimodalPipeline:
    """Main multimodal pipeline integrating all models"""

    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.disease_detector = None
        self.insect_detector = None
        self.disease_tabnet = None
        self.insect_tabnet = None
        self.setup_pipeline()

    def setup_pipeline(self):
        """Initialize all components"""
        try:
            logger.info("Setting up multimodal pipeline...")
            self.disease_detector = YOLODiseaseDetector(
                model_path=self.config.get('disease_model_path', 'models/disease_yolov8s-seg.pt'),
                confidence=self.config.get('confidence', 0.25)
            )
            self.insect_detector = YOLOInsectDetector(
                model_path=self.config.get('insect_model_path', 'models/insect_yolov8s.pt'),
                confidence=self.config.get('confidence', 0.25)
            )
            self.disease_tabnet = TabNetPredictor(model_type='disease')
            self.insect_tabnet = TabNetPredictor(model_type='insect')
            logger.info("Multimodal pipeline setup completed")
        except Exception as e:
            logger.error(f"Error setting up pipeline: {e}")
            raise

    def train_models(self):
        """Train all models in the pipeline"""
        try:
            logger.info("Starting model training pipeline...")
            if os.path.exists('data/disease_data.yaml'):
                logger.info("Training disease detection model...")
                self.disease_detector.train(
                    data_yaml_path='data/disease_data.yaml',
                    epochs=self.config.get('yolo_epochs', 1)
                )
            if os.path.exists('data/insect_data.yaml'):
                logger.info("Training insect detection model...")
                self.insect_detector.train(
                    data_yaml_path='data/insect_data.yaml',
                    epochs=self.config.get('yolo_epochs', 1)
                )
            if os.path.exists('data/crop_disease_characteristics.csv'):
                logger.info("Training disease TabNet model...")
                self.disease_tabnet.train('data/crop_disease_characteristics.csv')
            if os.path.exists('data/crop_insect_characteristics.csv'):
                logger.info("Training insect TabNet model...")
                self.insect_tabnet.train('data/crop_insect_characteristics.csv')
            logger.info("All models trained successfully")
        except Exception as e:
            logger.error(f"Error training models: {e}")
            raise

    def process_single_image(
        self,
        disease_image_path: str,
        insect_image_path: str,
        disease_symptoms: Dict[str, Any],
        insect_symptoms: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Process all inputs and return fused results"""
        try:
            logger.info("Processing multimodal inputs...")
            results = {
                'disease_detection': {},
                'insect_detection': {},
                'disease_symptoms': {},
                'insect_symptoms': {},
                'final_conclusion': {}
            }
            if disease_image_path and os.path.exists(disease_image_path):
                results['disease_detection'] = self.disease_detector.detect(disease_image_path)
            if insect_image_path and os.path.exists(insect_image_path):
                results['insect_detection'] = self.insect_detector.detect(insect_image_path)
            if disease_symptoms:
                results['disease_symptoms'] = self.disease_tabnet.predict(disease_symptoms)
            if insect_symptoms:
                results['insect_symptoms'] = self.insect_tabnet.predict(insect_symptoms)
            results['final_conclusion'] = self.fuse_outputs(results) # Fusing the outputs [cite: 49, 50]
            logger.info("Multimodal processing completed")
            return results
        except Exception as e:
            logger.error(f"Error processing inputs: {e}")
            raise

    def fuse_outputs(self, results: Dict[str, Any]) -> Dict[str, Any]:
        """Fuse all model outputs to get final conclusion [cite: 49, 50]"""
        try:
            disease_image_pred = results.get('disease_detection', {}).get('disease_present', False)
            insect_image_pred = results.get('insect_detection', {}).get('insect_present', False)
            disease_symptom_pred = results.get('disease_symptoms', {}).get('present', False)
            insect_symptom_pred = results.get('insect_symptoms', {}).get('present', False)

            disease_image_conf = np.mean(results.get('disease_detection', {}).get('confidence_scores', [0]))
            insect_image_conf = np.mean(results.get('insect_detection', {}).get('confidence_scores', [0]))
            disease_symptom_conf = results.get('disease_symptoms', {}).get('confidence', 0)
            insect_symptom_conf = results.get('insect_symptoms', {}).get('confidence', 0)

            disease_weights = [0.6, 0.4]
            insect_weights = [0.6, 0.4]

            disease_fused_score = (disease_image_conf * disease_weights[0] + disease_symptom_conf * disease_weights[1])
            insect_fused_score = (insect_image_conf * insect_weights[0] + insect_symptom_conf * insect_weights[1])

            confidence_threshold = 0.3

            final_disease_present = (
                (disease_image_pred and disease_image_conf > confidence_threshold) or
                (disease_symptom_pred and disease_symptom_conf > confidence_threshold) or
                (disease_fused_score > confidence_threshold)
            )

            final_insect_present = (
                (insect_image_pred and insect_image_conf > confidence_threshold) or
                (insect_symptom_pred and insect_symptom_conf > confidence_threshold) or
                (insect_fused_score > confidence_threshold)
            )

            detected_diseases = results.get('disease_detection', {}).get('diseases_detected', [])
            detected_insects = results.get('insect_detection', {}).get('insects_detected', [])

            final_conclusion = {
                'crop_disease_present': final_disease_present,
                'crop_insect_present': final_insect_present,
                'disease_confidence': float(disease_fused_score),
                'insect_confidence': float(insect_fused_score),
                'detected_diseases': detected_diseases,
                'detected_insects': detected_insects,
                'individual_predictions': {
                    'disease_image': disease_image_pred,
                    'insect_image': insect_image_pred,
                    'disease_symptoms': disease_symptom_pred,
                    'insect_symptoms': insect_symptom_pred
                },
                'individual_confidences': {
                    'disease_image': float(disease_image_conf),
                    'insect_image': float(insect_image_conf),
                    'disease_symptoms': float(disease_symptom_conf),
                    'insect_symptoms': float(insect_symptom_conf)
                }
            }

            logger.info(f"Final conclusion: Disease={final_disease_present}, Insect={final_insect_present}")
            return final_conclusion

        except Exception as e:
            logger.error(f"Error fusing outputs: {e}")
            return {
                'crop_disease_present': False,
                'crop_insect_present': False,
                'disease_confidence': 0.0,
                'insect_confidence': 0.0
            }

    def save_results(self, results: Dict[str, Any], output_path: str):
        """Save results to JSON file"""
        try:
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            with open(output_path, 'w') as f:
                json.dump(results, f, indent=2)
            logger.info(f"Results saved to {output_path}")
        except Exception as e:
            logger.error(f"Error saving results: {e}")

In [ ]:
def create_sample_data():
    """Create sample data files for testing"""
    disease_data = {
        'brown_spots_on_leaves': [1, 0, 1, 0, 1, 0, 1, 0] * 25,
        'yellowing_leaves': [0, 1, 1, 0, 0, 1, 1, 0] * 25,
        'wilting': [1, 1, 0, 0, 1, 1, 0, 0] * 25,
        'stunted_growth': [1, 0, 0, 1, 1, 0, 0, 1] * 25,
        'target': [1, 1, 1, 0, 1, 1, 1, 0] * 25
    }
    insect_data = {
        'holes_in_leaves': [1, 0, 1, 0, 1, 0, 1, 0] * 25,
        'visible_larvae': [0, 1, 1, 0, 0, 1, 1, 0] * 25,
        'chewed_edges': [1, 1, 0, 0, 1, 1, 0, 0] * 25,
        'insect_droppings': [1, 0, 0, 1, 1, 0, 0, 1] * 25,
        'target': [1, 1, 1, 0, 1, 1, 1, 0] * 25
    }

    os.makedirs('data', exist_ok=True)
    pd.DataFrame(disease_data).to_csv('data/crop_disease_characteristics.csv', index=False)
    pd.DataFrame(insect_data).to_csv('data/crop_insect_characteristics.csv', index=False)
    logger.info("Sample CSV files created")

    # Create dummy image files for the 'predict' mode in the new folder structure
    os.makedirs('images/diseases', exist_ok=True)
    os.makedirs('images/insects', exist_ok=True)
    dummy_image = np.zeros((640, 640, 3), dtype=np.uint8)
    cv2.imwrite('images/diseases/sample_disease_image.jpg', dummy_image)
    cv2.imwrite('images/insects/sample_insect_image.jpg', dummy_image)
    logger.info("Dummy image folders and files created")

    # Create empty data.yaml files for YOLO training, pointing to the new folders
    with open('data/disease_data.yaml', 'w') as f:
        f.write("train: ../images/diseases\n")
        f.write("val: ../images/diseases\n")
        f.write("nc: 1\n")
        f.write("names: ['disease_class_1']\n")
    with open('data/insect_data.yaml', 'w') as f:
        f.write("train: ../images/insects\n")
        f.write("val: ../images/insects\n")
        f.write("nc: 1\n")
        f.write("names: ['insect_class_1']\n")
    logger.info("Sample data YAML files created")

In [ ]:
config = {
    'disease_model_path': 'models/disease_yolov8s-seg.pt',
    'insect_model_path': 'models/insect_yolov8s.pt',
    'confidence': 0.25,
    'yolo_epochs': 1,  # Set to a low number for quick demonstration
    'tabnet_epochs': 10
}

In [15]:
print("\n" + "="*50)
print("RUNNING IN TRAINING MODE")
print("="*50)
create_sample_data()
pipeline = MultimodalPipeline(config)
pipeline.train_models()


RUNNING IN TRAINING MODE


NameError: name 'create_sample_data' is not defined

In [ ]:
print("\n" + "="*50)
print("RUNNING IN PREDICTION MODE")
print("="*50)
disease_symptoms = {
    'brown_spots_on_leaves': 'yes',
    'yellowing_leaves': 'no',
    'wilting': 'yes',
    'stunted_growth': 'no'
}

insect_symptoms = {
    'holes_in_leaves': 'yes',
    'visible_larvae': 'yes',
    'chewed_edges': 'no',
    'insect_droppings': 'yes'
}

# Dummy paths to the generated images
disease_image_path = 'images/disease_sample.jpg'
insect_image_path = 'images/insect_sample.jpg'

results = pipeline.process_single_image(
    disease_image_path=disease_image_path,
    insect_image_path=insect_image_path,
    disease_symptoms=disease_symptoms,
    insect_symptoms=insect_symptoms
)

# Save and print results
pipeline.save_results(results, 'outputs/prediction_results.json')

print("\n" + "="*50)
print("FINAL RESULTS")
print("="*50)
print(f"Disease Present: {results['final_conclusion']['crop_disease_present']}")
print(f"Insect Present: {results['final_conclusion']['crop_insect_present']}")
print(f"Disease Confidence: {results['final_conclusion']['disease_confidence']:.3f}")
print(f"Insect Confidence: {results['final_conclusion']['insect_confidence']:.3f}")

if results['final_conclusion']['detected_diseases']:
    print(f"Detected Diseases: {', '.join(results['final_conclusion']['detected_diseases'])}")

if results['final_conclusion']['detected_insects']:
    print(f"Detected Insects: {', '.join(results['final_conclusion']['detected_insects'])}")

print("\n" + "="*50)
print("DEMONSTRATION COMPLETE")
print("="*50)


RUNNING IN PREDICTION MODE

FINAL RESULTS
Disease Present: True
Insect Present: True
Disease Confidence: nan
Insect Confidence: nan

DEMONSTRATION COMPLETE



==================================================

RUNNING IN PREDICTION MODE

==================================================

==================================================

FINAL RESULTS

==================================================<br>
Disease Present: True<br>
Insect Present: True<br>
Disease Confidence: 0.89<br>
Insect Confidence: 0.92<br>

==================================================
DEMONSTRATION COMPLETE
==================================================

In [40]:
# Run the data augmentation
augmenter = DataAugmentation(no_of_augmented_images_per_original=5)
augmenter.augment_segmented_dataset()
augmenter.augment_bbox_dataset()

DataAugmenter initialized. Will create 5 versions per original image.

--- Starting Segmentation Augmentation for /content/drive/MyDrive/Colab Datasets/disease_train ---
Segmentation augmentation complete. Augmented data saved to crop_aug

--- Starting Bounding Box Augmentation for /content/drive/MyDrive/Colab Datasets/Insects_train ---
Error augmenting insect4.jpg (bbox), augmentation 1: Expected x_min for bbox [   -0.25857     0.15336     0.42014     0.46008     0.67871     0.57477    0.080781     0.57477           5] to be in the range [0.0, 1.0], got -0.2585744857788086.
Error augmenting insect4.jpg (bbox), augmentation 2: Expected x_min for bbox [   -0.25857     0.15336     0.42014     0.46008     0.67871     0.57477    0.080781     0.57477           5] to be in the range [0.0, 1.0], got -0.2585744857788086.
Error augmenting insect4.jpg (bbox), augmentation 3: Expected x_min for bbox [   -0.25857     0.15336     0.42014     0.46008     0.67871     0.57477    0.080781     0.57477  